# Introduction to NLP fundamentals in TensorFlow

NLP has the goal of deriving information out of natural language (could be sequences of text or speech).

Another common term for NLP problem is sequence to sequence problems (seq2seq).

## Check for GPU

In [1]:
!nvidia-smi
!nvidia-smi -L

Fri Jul  8 20:14:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 516.59       Driver Version: 516.59       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   39C    P5    16W /  N/A |    477MiB /  6144MiB |     32%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Get helper functions

In [2]:
import os
if not os.path.exists("helper_functions.py"):
    !python -m wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

## Get a text dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as disaster or not disaster).

See the original source: https://www.kaggle.com/code/jhoward/getting-started-with-nlp-for-absolute-beginners/notebook

In [3]:
if not os.path.exists("nlp_getting_started.zip"):
    !python -m wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
unzip_data("C:/Selbststudium/Udemy/Udemy_TensorFlow_Certificate/nlp_getting_started.zip")

## Visualizing a text dataset

To visualize our text samples, we first have to read them in , one way to do so would be to use Python: https://realpython.com/read-write-files-python/ 

But i prefer to get visual straight away.

So another way to do this is to use pandes...

In [5]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [9]:
train_df["text"][1], train_df["target"][1]

('Forest fire near La Ronge Sask. Canada', 1)

In [11]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


# What does the test dataframe look like?
test_df.head()

In [14]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [15]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [26]:
# Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5) # Create random indexes not higher than the total number
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
    _, text, target = row
    print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
    print(f"Text: {text}\n")

Target: 0 (not real disaster)
Text: her cake looks like a mudslide hah

Target: 0 (not real disaster)
Text: OMEGA MEN Writer Explores Terrorism Religion In an 'Epic KYLE RAYNER Story' https://t.co/utc7pdIdfo via @Newsarama

Target: 1 (real disaster)
Text: @Haji_Hunter762 @MiddleEastEye maybe some muzzies will spontaneously combust! It's like Allah suicide bombing them as payback ????????

Target: 1 (real disaster)
Text: @wocowae Police Officer Wounded Suspect Dead After Exchanging Shots http://t.co/oiOeCbsh1f ushed

Target: 1 (real disaster)
Text: When there's a fire alarm going off in zehrs and we keep working for 20 minutes then decide to evacuate everyone..



### Split data into training and validation sets

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
# Use train_test_split to split training data into training and validation sets 
train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    train_df_shuffled["text"].to_numpy(),
    train_df_shuffled["target"].to_numpy(),
    test_size=0.1,
    random_state=42)

In [29]:
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

In [30]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Cnverting text into numbers

When dealing with a text problem, one of the first thing you'll have to do before you can build a model is to convert your text to numbers. 

There are a few ways to do this, namely:
* Tokenization - direct mapping of token (a token could be a word or a character)
* Embedding - create a matrix of feature vector for each token (the size of the feature vector can be defined and this embedding can be learned)

### Text vectorization (tokenization)

In [32]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [39]:
train_sentences[0].split(), len(train_sentences[0].split())

(['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet'], 7)

In [52]:
# Find the average number of tokes (words) in the training tweets
max_length = round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))
max_length

15

In [49]:
max_vocab_length = 10000

In [53]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(
    max_tokens=max_vocab_length,
    standardize="lower_and_strip_punctuation", # Default
    split="whitespace", # Default
    ngrams=None, # Default
    output_mode="int",
    output_sequence_length=max_length,
    pad_to_max_tokens=True) # Default

In [54]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [56]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]], dtype=int64)>

In [63]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\n\
    Vectorized version:")
text_vectorizer([random_sentence])

Original text:
FedEx no longer to transport bioterror germs in wake of anthrax lab mishaps http://t.co/HQsU8LWltH via @usatoday
    Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 577,   40,  600,    5,  832,  609, 1082,    4,  722,    6, 1096,
         740, 1122,    1,   49]], dtype=int64)>

In [68]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
len(words_in_vocab), top_5_words, bottom_5_words

(10000,
 ['', '[UNK]', 'the', 'a', 'in'],
 ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1'])

### Creating an Embedding using an EmbeddingLayer

To make our embedding, we're going to use TensorFlow's EmbeddingLayer: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

The parameters we care most about for our embedding layer: 
* `input_dim` = the size of our vocabulary
* `output_dim` = the size of the output embedding vector, for example, a value of 100 would mean each tolen gets represented by a vector 100 long
* `input_length` = length of the sequences being passed to the embedding layer 

In [71]:
from tensorflow.keras import layers

embedding = layers.Embedding(
    input_dim=max_vocab_length,
    output_dim=128,
    input_length=max_length)
embedding

In [79]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
    \nEmbedded version: ")

# Embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
Suicide bomber kills 15 in Saudi security site mosque - Reuters Canada: Reuters CanadaSuicide bomber kills 15 ... http://t.co/Ktd5IG9M5o    
Embedded version: 


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.04711789, -0.0050141 , -0.01471903, ..., -0.00565889,
         -0.04793103, -0.02626393],
        [ 0.01950938, -0.0002872 , -0.0345827 , ..., -0.00772265,
         -0.04735669, -0.04657718],
        [-0.00798779,  0.02142051,  0.04441834, ..., -0.03355129,
          0.02662284, -0.02096142],
        ...,
        [-0.0266984 , -0.00469609, -0.02429711, ...,  0.03633272,
          0.02959866,  0.02883315],
        [ 0.01950938, -0.0002872 , -0.0345827 , ..., -0.00772265,
         -0.04735669, -0.04657718],
        [-0.00798779,  0.02142051,  0.04441834, ..., -0.03355129,
          0.02662284, -0.02096142]]], dtype=float32)>

In [81]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.04711789, -0.0050141 , -0.01471903,  0.01304183, -0.04146459,
        -0.03157397,  0.01970831,  0.0202979 , -0.04443971,  0.00995062,
         0.00340787,  0.0275279 ,  0.04998341, -0.00753515, -0.03382881,
        -0.0356412 ,  0.00628664,  0.00704982,  0.03043001, -0.02376068,
        -0.04893368, -0.02752771,  0.0421307 ,  0.04174635,  0.02783854,
        -0.0134859 , -0.01976111,  0.04499428,  0.02015338,  0.01906024,
        -0.0222254 , -0.02693412,  0.01043129, -0.00071161,  0.02614144,
         0.0296798 , -0.02180864, -0.04273058,  0.04140014, -0.0297562 ,
        -0.03618004, -0.00546589, -0.04739288, -0.03043246,  0.01205875,
        -0.03058838, -0.01398017,  0.01896108,  0.04784377, -0.02443922,
         0.01360283,  0.02342445, -0.02915647,  0.03443031,  0.02965141,
         0.0184122 ,  0.01180498,  0.03877083,  0.04882226,  0.0011472 ,
        -0.00348576, -0.04566009, -0.03550897, -0.01876332, -0.04826217,
  

## Modelling a text dataset (running a series of experiments)

Now we've got a way to turn our text sequences into numbers, it's time  to start building a series of modelling experiments.

We'll start with a baseline and move on from there.

* Model 0: Naive Bayes (baseline), from sklearn ML map: 
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM (RNN)
* Model 3: GRU (RNN) 
* Model 4: Bidirectional-LSTM model (RNN)
* Model 5: 1D Convolutional Network (CNN)
* Model 6: TensorFlow Hub Pretrained Feature Extractor (using transfer learning for NLP)
* Model 7: Same as model 6 with 10% of training data

How are we going to approach all of these?

Use the standard steps in modelling with tensorflow:
* Create a model
* Build the model
* Fit the model
* Evaluate the model

### Model 0: Getting a baseline

As with all machine learning modelling experiments it's important to create a baseline model so you've got a benchmark for future experiments to build uopn.

To create our baseline, we'll use Sklearn's Mulinomial Naive Bayes using the TF-IDF formula to convert our words to numbers.

> 🔑 **Note:** It's common to use non DL-algorithms as a baseline because of their speed and then later using DL to see if you can improve upon them.

In [85]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([ # Pipeline is similar to tensorflow's sequential
    ("tfidf", TfidfVectorizer()), # name of the step is "tfidf"; convert words to numbers using tfidf
    ("clf", MultinomialNB()), # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [88]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [91]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
      dtype=int64)

In [92]:
train_labels

array([0, 0, 1, ..., 1, 1, 0], dtype=int64)

### Creating an evaluation function for our model experiments

We could evaluate all of our model's predictions with different metrics every time, however this will be cumbersome and could easily be fixed with a function.

Let's create one to compare our model's predictions with the truth labels using the following metrics:
* Accuracy
* Precision
* Recall
* F1-score

For a deep overview of many different methods, see the Sklearn documentation.

In [94]:
# Function to evaluate accuracy, precision, reall and f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
    """
    Calculates model accuracy, precision, recall and f1 score of a binare classification model.
    """
    # Calculate model accuracy
    model_accuracy = accuracy_score(y_true, y_pred) * 100
    # Calculate model precision, recall and f1-score using "weighted" average
    model_precision, model_recall, model_f1, _, = precision_recall_fscore_support(y_true, y_pred, average="weighted")
    model_results = {
        "accuracy": model_accuracy, 
        "precision": model_precision,
        "recall": model_recall, 
        "f1-score": model_f1}
    return model_results

In [95]:
# Get baseline results
baseline_results = calculate_results(
    y_true=val_labels,
    y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1-score': 0.7862189758049549}